##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Treinamento multiworker com o Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este tutorial demonstra como fazer um treinamento distribuído multiworker com um modelo do Keras e a API `Model.fit` usando a API `tf.distribute.MultiWorkerMirroredStrategy`. Com a ajuda desta estratégia, um modelo do Keras desenvolvido para executar em um único worker pode funcionar de forma transparente em diversos workers, com alterações de código mínimas.

Para saber como usar `MultiWorkerMirroredStrategy` com o Keras e um loop de treinamento personalizado, confira [Loop de treinamento personalizado com o Keras e MultiWorkerMirroredStrategy](multi_worker_with_ctl.ipynb).

Este tutorial contém um exemplo mínimo de multiworker, com dois workers, para fins de demonstração.

### Escolha da estratégia certa

Antes de começar, confirme se `tf.distribute.MultiWorkerMirroredStrategy` é a escolha certa para seu treinamento e aceleradores. Existem duas formas comuns de distribuir o treinamento com paralelismo de dados:

- *Treinamento síncrono*, em que os passos do treinamento são sincronizados nos workers e nas réplicas, como `tf.distribute.MirroredStrategy`, `tf.distribute.TPUStrategy` e `tf.distribute.MultiWorkerMirroredStrategy`. Todos os workers são treinados com diferentes partes dos dados de entrada em sincronia, agregando os gradientes em cada passo.
- *Treinamento assíncrono*, em que os passos de treinamento não são sincronizados estritamente, como `tf.distribute.experimental.ParameterServerStrategy`. Todos os workers são treinados de forma independente com os dados de entrada, atualizando as variáveis de forma assíncrona.

Se você deseja fazer um treinamento multiworker síncrono sem TPU, então `tf.distribute.MultiWorkerMirroredStrategy` é a sua escolha. Ele cria cópias de todas as variáveis nas camadas do modelo em cada dispositivo, em todos os workers. Ele usa `CollectiveOps`, uma operação do TensorFlow para comunicação coletiva, para agregar os gradientes e manter as variáveis sincronizadas. Se você tiver interesse, confira o parâmetro `tf.distribute.experimental.CommunicationOptions` para ver as opções de implementação de coletivos.

Confira uma visão geral das APIs `tf.distribute.Strategy` em [Treinamento distribuído no TensorFlow](../../guide/distributed_training.ipynb).

## Configuração

Comece com as importações necessárias:

In [ ]:
import json
import os
import sys

Antes de importar o TensorFlow, faça algumas alterações no ambiente:

- Em uma aplicação real, cada worker estaria em uma máquina diferente. Para a finalidade deste tutorial, todos os workers serão executados **nesta** máquina. Portanto, desative todas as GPUs para evitar erros causados quando todos os workers tentam usar a mesma GPU.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

- Redefina a variável de ambiente `TF_CONFIG` (você verá mais detalhes sobre isso posteriormente):

In [ ]:
os.environ.pop('TF_CONFIG', None)

- O diretório atual deve estar no caminho do Python. Dessa forma, o notebook pode importar os arquivos escritos por `%%writefile` posteriormente:


In [ ]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

Instale `tf-nightly`, pois a frequência de salvamento de checkpoints em um determinado passo com o argumento `save_freq` em `tf.keras.callbacks.BackupAndRestore` foi lançada no TensorFlow 2.10:

In [ ]:
!pip install tf-nightly

Por fim, importe o TensorFlow:

In [ ]:
import tensorflow as tf

### Definição do dataset e do modelo

Agora, crie um arquivo `mnist_setup.py` com uma configuração simples de modelo e dataset. Este arquivo do Python será usado pelos processos dos workers neste tutorial:

In [ ]:
%%writefile mnist_setup.py

import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # The `x` arrays are in uint8 and have values in the [0, 255] range.
  # You need to convert them to float32 with values in the [0, 1] range.
  x_train = x_train / np.float32(255)
  y_train = y_train.astype(np.int64)
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000).repeat().batch(batch_size)
  return train_dataset

def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
      metrics=['accuracy'])
  return model

### Treinamento do modelo em um único worker

Tente treinar um modelo com um pequeno número de épocas e observe os resultados de um *único worker* para garantir que tudo esteja funcionando corretamente. À medida que o treinamento prosseguir, a perda deve cair e a exatidão deve aumentar.

In [ ]:
import mnist_setup

batch_size = 64
single_worker_dataset = mnist_setup.mnist_dataset(batch_size)
single_worker_model = mnist_setup.build_and_compile_cnn_model()
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)

## Configuração multiworker

Agora, vamos entrar na seara do treinamento multiworker.

### Cluster com trabalhos e tarefas

No TensorFlow, o treinamento distribuído envolve um `'cluster'` com diversos trabalhos, e cada trabalho pode ter uma ou mais `'task'`s (tarefas).

Você precisará da variável de ambiente de configuração `'TF_CONFIG'` para fazer o treinamento em várias máquinas, em que cada uma pode ter uma função diferente. A variável `TF_CONFIG` é uma string JSON usada pra especificar a configuração de cluster em cada worker que faz parte do cluster.

Existem dois componentes da variável `TF_CONFIG`: `'cluster'` e `'task'` (tarefa).

- `'cluster'` é igual para todos os workers e fornece informações sobre o cluster de treinamento, que é um dicionário consistindo de diferentes tipos de trabalhos, como `'worker'` ou `'chief'` (principal).

    - No treinamento multiworker com `tf.distribute.MultiWorkerMirroredStrategy`, geralmente existe um `'worker'` que tem mais responsabilidades, como salvar um checkpoint e escrever um arquivo de resumo para o TensorBoard, além das tarefas de um `'worker'` comum. Esse `'worker'` é chamado de chamado de principal (com o nome de trabalho `'chief'`).
    - É comum que o worker com `'index'` (índice) `0` seja o `'chief'`.

- `'task'` (tarefa) fornece informações sobre a tarefa atual e é diferente em cada worker. Ela especifica o `'type'` (tipo) e o `'index'` (índice) desse worker.

Veja um exemplo de configuração abaixo:

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

Observe que `tf_config` é apenas uma variável local no Python. Para usá-la na configuração do treinamento, serialize-a como JSON e coloque-a em uma variável de ambiente `TF_CONFIG`.

In [ ]:
json.dumps(tf_config)

No exemplo de configuração acima, você define o `'type'` da tarefa como `'worker'` e o `'index'` da tarefa como `0`. Portanto, essa máquina é o *primeiro* worker. Ela será designada com o worker `'chief'`.

Observação: também será preciso definir a variável de ambiente `TF_CONFIG` nas outras máquinas, e ela deve ter o mesmo dicionário `'cluster'`, mas um `'type'` de tarefa ou um `'index'` de tarefa diferente, dependendo das funções das máquinas.

Na prática, você criaria diversos workers em endereços IP/portas externos e definiria a variável `TF_CONFIG` em cada worker da forma adequada. Para fins de ilustração, este tutorial mostra como definir uma variável `TF_CONFIG` com dois workers no `localhost`:

- A variável `'TF_CONFIG'` do primeiro worker (`'chief'`) é exibida acima.
- Para o segundo worker, defina `tf_config['task']['index']=1`

### Variáveis de ambiente e subprocessos nos notebooks

Os subprocessos herdam as variáveis de ambiente do pai. Portanto, se você definir uma variável de ambiente neste processo do Jupyter Notebook:

In [ ]:
os.environ['GREETINGS'] = 'Hello TensorFlow!'

...você poderá acessar a variável de ambiente nos subprocessos:

In [ ]:
%%bash
echo ${GREETINGS}

Na próxima seção, você usará esse método para passar a variável `'TF_CONFIG'` aos subprocessos do worker. Em um cenário real, você jamais iniciaria os trabalho dessa forma, mas este tutorial apenas mostra como fazê-lo em um exemplo mínimo de multiworker.

## Treinamento do modelo

Para treinar o modelo, primeiro crie uma instância de `tf.distribute.MultiWorkerMirroredStrategy`:

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

Observação: `TF_CONFIG` é processada e os servidores gRPC do TensorFlow são iniciados no momento em que `MultiWorkerMirroredStrategy` é chamado, então a variável de ambiente `TF_CONFIG` precisa ser definida antes da criação de uma instância de `tf.distribute.Strategy`. Como `TF_CONFIG` ainda não foi definida, a estratégia acima é efetivamente um treinamento em um único worker.

Com a integração da API `tf.distribute.Strategy` com `tf.keras`, a única mudança necessária para distribuir o treinamento em vários workers é encapsular a criação do modelo e a chamada a `model.compile()` dentro de `strategy.scope()`. O escopo da estratégia de distribuição dita como e onde as variáveis são criadas. No caso de `MultiWorkerMirroredStrategy`, as variáveis criadas são `MirroredVariables` e são replicadas em cada um dos workers.


In [ ]:
with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()

Observação: atualmente, há uma limitação do `MultiWorkerMirroredStrategy`, em que as operações do TensorFlow precisam ser criadas após a criação da instância da estratégia. Se você se deparar com `RuntimeError: Collective ops must be configured at program startup` (Erro em tempo de execução: operações de coletivos precisam ser configuradas na inicialização do programa), tente criar a instância de `MultiWorkerMirroredStrategy` no começo do programa e coloque o código que pode criar operações após a criação da instância da estratégia.

Para fazer a execução com `MultiWorkerMirroredStrategy`, você precisará executar processos dos workers e passar uma variável `TF_CONFIG` para eles.

Como o arquivo `mnist_setup.py` escrito anteriormente, aqui está o arquivo `main.py` que cada worker executará:

In [ ]:
%%writefile main.py

import os
import json

import tensorflow as tf
import mnist_setup

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

strategy = tf.distribute.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = mnist_setup.mnist_dataset(global_batch_size)

with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()


multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)

No trecho de código acima, observe que `global_batch_size`, que é passado para `Dataset.batch`, é definido como `per_worker_batch_size * num_workers`. Dessa forma, garante-se que cada worker processe lotes de `per_worker_batch_size` exemplos, não importa o número de workers.

Agora, o diretório atual contém os dois arquivos do Python:

In [ ]:
%%bash
ls *.py

Serialize a variável `TF_CONFIG` como JSON e adicione-a às variáveis de ambiente:

In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Agora, você pode iniciar um processo de worker, que executará `main.py` e usará a variável `TF_CONFIG`:

In [ ]:
# first kill any previous runs
%killbgscripts

In [ ]:
%%bash --bg
python main.py &> job_0.log

Veja algumas observações sobre o comando acima:

1. Ele usa `%%bash`, que é um [“magic” dos notebooks](https://ipython.readthedocs.io/en/stable/interactive/magics.html) que executa alguns comandos bash.
2. Ele usa o sinalizador `--bg` para executar o processo `bash` em segundo plano, pois esse worker não será encerrado. Ele aguarda todos os workers antes de iniciar.

O processo do worker em segundo plano não exibe a saída nesse notebook. `&>` redireciona a saída para um arquivo para que você possa analisar o que aconteceu.

Aguarde alguns segundos para o processo iniciar:

In [ ]:
import time
time.sleep(10)

Agora, verifique a saída no arquivo de log do worker:

In [ ]:
%%bash
cat job_0.log

A última linha do arquivo de log deve dizer: `Started server with target: grpc://localhost:12345` (Serviço iniciado com destino: grpc://localhost:12345). Agora, o primeiro worker está pronto e aguarda que todos os outros workers estejam prontos para prosseguir.

Atualize `tf_config` para o processo do segundo worker prosseguir:

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Inicie o segundo worker. O treinamento será iniciado, já que todos os workers estão ativos (então não há necessidade de colocar esse processo em segundo plano):

In [ ]:
%%bash
python main.py

Se você verificar novamente os logs escritos pelo primeiro worker, notará que ele participou do treinamento do modelo:

In [ ]:
%%bash
cat job_0.log

Observação: a execução poderá ser mais lenta do que a execução de teste no começo deste tutorial porque executar diversos workers em uma única máquina adiciona uma sobrecarga. O objetivo não é melhorar o tempo de treinamento, mas dar um exemplo de treinamento multiworker.


In [ ]:
# Delete the `TF_CONFIG`, and kill any background tasks so they don't affect the next section.
os.environ.pop('TF_CONFIG', None)
%killbgscripts

## Treinamento multiworker aprofundado


Até agora, você aprendeu como fazer uma configuração multiworker básica. O restante do tutorial aborda em detalhes outros fatores, que podem ser úteis ou importantes para casos de uso reais.

### Fragmentação do dataset

No treinamento multiworker, a *fragmentação do dataset* é necessária para convergência e desempenho.

O exemplo da seção anterior usa a fragmentação automática padrão da API `tf.distribute.Strategy`. Você também pode alterar a fragmentação definindo a `tf.data.experimental.AutoShardPolicy` de `tf.data.experimental.DistributeOptions`.

Para saber mais sobre *fragmentação automática*, confira o [guia Entrada distribuída](https://www.tensorflow.org/tutorials/distribute/input#sharding).

Veja um exemplo rápido de como desativar a fragmentação automática para que cada réplica processe todos os exemplos (*não recomendado*):


In [ ]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

global_batch_size = 64
multi_worker_dataset = mnist_setup.mnist_dataset(batch_size=64)
dataset_no_auto_shard = multi_worker_dataset.with_options(options)

### Avaliação

Se você também passar `validation_data` para `Model.fit`, ele alternará entre treinamento e avaliação para cada época. O trabalho de avaliação é distribuído entre o mesmo conjunto de workers, e os resultados são agregados e disponibilizados para todos os workers.

De maneira similar ao treinamento, o dataset de validação é fragmentado automaticamente no nível de arquivo. Você precisa definir o tamanho global de lote no dataset de validação e também `validation_steps`.

Recomenda-se um dataset repetido (fazendo uma chamada a `tf.data.Dataset.repeat`) para a avaliação.

Outra opção é criar outra tarefa que lê periodicamente os checkpoints e executa a avaliação. É isso o que um Estimador faz. Mas esse método não é recomendado para fazer a avaliação e, portanto, os detalhes foram omitidos.

### Desempenho

Para ajustar o desempenho do treinamento multiworker, você pode fazer o seguinte:

- O `MultiWorkerMirroredStrategy` tem diversas [implementações de comunicação de coletivos](https://www.tensorflow.org/api_docs/python/tf/distribute/experimental/CommunicationImplementation):

    - O `RING` usa coletivos com base em anel usando o gRPC como a camada de comunicação entre hosts.
    - O `NCCL` usa a [Biblioteca de Comunicação Coletiva da NVIDIA](https://developer.nvidia.com/nccl) para implementar coletivos.
    - `AUTO` adia a escolha para o runtime.

    A melhor escolha de implementação de coletivos depende do número e tipo de GPUs, além das interconexões de rede no cluster. Para sobrescrever a escolha automática, especifique o parâmetro `communication_options` do construtor de `MultiWorkerMirroredStrategy`. Por exemplo:

    ```python
    communication_options=tf.distribute.experimental.CommunicationOptions(implementation=tf.distribute.experimental.CommunicationImplementation.NCCL)
    ```

- Converta as variáveis para <code>tf.float</code>, se possível:

    - O modelo oficial ResNet inclui [um exemplo](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466) de como fazer a conversão.

### Tolerância a falhas

No treinamento síncrono, haveria falha no cluster se um dos workers falhasse e não houvesse nenhum mecanismo de recuperação em caso de falhas.

O uso do Keras com `tf.distribute.Strategy` tem a vantagem de tolerância a falhas em casos quando os workers param de funcionar ou ficam instáveis. Para fazer isso, basta preservar o estado do treinamento no sistema de arquivos distribuído de sua escolha de tal forma que, quando uma instância que falhou ou foi interrompida anteriormente reiniciar, o estado de treinamento seja recuperado.

Quando um worker ficar indisponível, outros workers falharão (possivelmente após um tempo limite esgotado). Nesses casos, o worker indisponível precisa ser reiniciado, assim como os outros workers que falharam.

Observação: anteriormente, o callback `ModelCheckpoint` fornecia um mecanismo para restaurar o estado do treinamento ao reiniciar após a falha de um trabalho para o treinamento multiworker. A equipe do TensorFlow está lançando um novo callback [`BackupAndRestore`](#scrollTo=kmH8uCUhfn4w), que também acrescenta suporte ao treinamento em um único worker para que a experiência seja consistente. Além disso, a funcionalidade de tolerância a falhas foi removida do callback `ModelCheckpoint` existente. A partir de agora, as aplicações que dependem desse comportamento devem migrar para o novo callback `BackupAndRestore`.

#### Callback `ModelCheckpoint`

O callback `ModelCheckpoint` não oferece mais a funcionalidade de tolerância a falhas. Use o callback [`BackupAndRestore`](#scrollTo=kmH8uCUhfn4w).

O callback `ModelCheckpoint` ainda pode ser usado para salvar checkpoints. Mas, se o treinamento tiver sido interrompido ou concluído com êxito, o usuário será responsável por carregar o modelo manualmente para continuar o treinamento a partir do checkpoint.

Opcionalmente, os usuários podem optar por salvar e restaurar o modelo/pesos fora do callback `ModelCheckpoint`.

### Como salvar e restaurar o modelo

Para salvar o modelo usando `model.save` ou `tf.saved_model.save`, o destino precisa ser diferente para cada worker.

- Para workers que não sejam chief, você precisará salvar o modelo em um diretório temporário.
- Para o worker chief, você precisará salvar no diretório do modelo fornecido.

Os diretórios temporários no worker precisam ser únicos para evitar erros em que diversos workers tentam escrever no mesmo local.

O modelo salvo em todos os diretórios é idêntico e, tipicamente, somente o modelo salvo pelo worker chief deve ser referenciado para restauração e serviço.

Você precisa ter uma lógica de limpeza que exclua os diretórios temporários criados pelos workers após a conclusão do treinamento.

O motivo para salvar no worker chief e nos outros ao mesmo tempo é porque você poderá agregar variáveis durante a criação de checkpoints, o que requer que tanto o worker chief quanto os outros participem do protocolo de comunicação de redução total (allreduce). Por outro lado, deixar o worker chief e os outros salvarem no mesmo diretório do modelo causará erros devido à contenção.

Ao usar `MultiWorkerMirroredStrategy`, o programa é executado em cada worker e, para saber se o worker atual é o chief, é usado o objeto resolvedor do cluster, que tem os atributos `task_type` e `task_id`:

- `task_type` indica o que o trabalho atual é (por exemplo, `'worker'`).
- `task_id` indica o identificador do worker.
- O worker com `task_id == 0` é designado como o worker chief.

No trecho de código abaixo, a função `write_filepath` fornece o caminho onde escrever o arquivo, que depende do `task_id` do worker:

- Para o worker chief (com `task_id == 0`), ela escreve no caminho de arquivo original.
- Para os outros workers, ela cria um diretório temporário, `temp_dir`, com o `task_id` no caminho de diretório onde o arquivo será escrito:

In [ ]:
model_path = '/tmp/keras-model'

def _is_chief(task_type, task_id):
  # Note: there are two possible `TF_CONFIG` configurations.
  #   1) In addition to `worker` tasks, a `chief` task type is use;
  #      in this case, this function should be modified to
  #      `return task_type == 'chief'`.
  #   2) Only `worker` task type is used; in this case, worker 0 is
  #      regarded as the chief. The implementation demonstrated here
  #      is for this case.
  # For the purpose of this Colab section, the `task_type` is `None` case
  # is added because it is effectively run with only a single worker.
  return (task_type == 'worker' and task_id == 0) or task_type is None

def _get_temp_dir(dirpath, task_id):
  base_dirpath = 'workertemp_' + str(task_id)
  temp_dir = os.path.join(dirpath, base_dirpath)
  tf.io.gfile.makedirs(temp_dir)
  return temp_dir

def write_filepath(filepath, task_type, task_id):
  dirpath = os.path.dirname(filepath)
  base = os.path.basename(filepath)
  if not _is_chief(task_type, task_id):
    dirpath = _get_temp_dir(dirpath, task_id)
  return os.path.join(dirpath, base)

task_type, task_id = (strategy.cluster_resolver.task_type,
                      strategy.cluster_resolver.task_id)
write_model_path = write_filepath(model_path, task_type, task_id)

Agora, está tudo pronto para salvar:

Obsoleto: para objetos do Keras, recomenda-se usar o novo formato de alto nível `.keras` e `tf.keras.Model.export`, conforme demonstrado [neste guia](https://www.tensorflow.org/guide/keras/save_and_serialize). O formato de baixo nível SavedModel continua com suporte para códigos existentes.

In [ ]:
multi_worker_model.save(write_model_path)

Conforme descrito acima, posteriormente, o modelo só deve ser carregado a partir do caminho de arquivo no qual o worker chief salvou. Portanto, remova os diretórios temporários nos quais os workers que não são chief salvaram:

In [ ]:
if not _is_chief(task_type, task_id):
  tf.io.gfile.rmtree(os.path.dirname(write_model_path))

Agora, quando for a hora de carregar, use a API conveniente `tf.keras.models.load_model` e prossiga com o trabalho.

Aqui, pressuponha o uso de um único worker para carregar e continuar o treinamento. Nesse caso, não faça uma chamada a `tf.keras.models.load_model` em outra função `strategy.scope()` (observe que `strategy = tf.distribute.MultiWorkerMirroredStrategy()`, conforme definido anteriormente):

In [ ]:
loaded_model = tf.keras.models.load_model(model_path)

# Now that the model is restored, and can continue with the training.
loaded_model.fit(single_worker_dataset, epochs=2, steps_per_epoch=20)

### Como salvar e restaurar checkpoints

Por outro lado, a criação de checkpoints permite salvar os pesos do modelo e restaurá-los sem precisar salvar o modelo inteiro.

Aqui, você criará um `tf.train.Checkpoint` que monitora o modelo, que é gerenciado por `tf.train.CheckpointManager` de forma que apenas o último checkpoint seja preservado:

In [ ]:
checkpoint_dir = '/tmp/ckpt'

checkpoint = tf.train.Checkpoint(model=multi_worker_model)
write_checkpoint_dir = write_filepath(checkpoint_dir, task_type, task_id)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=write_checkpoint_dir, max_to_keep=1)

Quando o `CheckpointManager` estiver configurado, você poderá salvar e remover os checkpoints que os workers que não são chief salvaram:

In [ ]:
checkpoint_manager.save()
if not _is_chief(task_type, task_id):
  tf.io.gfile.rmtree(write_checkpoint_dir)

Agora, quando você precisar restaurar o modelo, encontrará o último checkpoint salvo usando a função conveniente `tf.train.latest_checkpoint`. Após restaurar o checkpoint, você pode continuar o treinamento.

In [ ]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint.restore(latest_checkpoint)
multi_worker_model.fit(multi_worker_dataset, epochs=2, steps_per_epoch=20)

#### Callback `BackupAndRestore`

Para fornecer a funcionalidade de tolerância a falhas, o callback `tf.keras.callbacks.BackupAndRestore` faz backup do modelo e do estado atual do treinamento em um arquivo de checkpoint temporário no argumento `backup_dir` de `BackupAndRestore`.

Observação: no TensorFlow 2.9, é feito backup do modelo e estado do treinamento atuais após cada época. Na versão `tf-nightly` e a partir do TensorFlow 2.10, o callback `BackupAndRestore` pode fazer backup do modelo e estado do treinamento após cada época ou determinado número de passos. `BackupAndRestore` aceita um argumento opcional `save_freq`. `save_freq` aceita `'epoch'` ou um valor `int`. Se `save_freq` for definido como `'epoch'`, é feito backup do modelo após cada época. Se `save_freq` for definido como um valor inteiro maior do que `0`, é feito backup do modelo após cada `save_freq` lotes.

Quando os trabalhos são interrompidos e reiniciados, o callback `BackupAndRestore` restaura o último checkpoint, e você pode continuar o treinamento pelo começo de cada época e passo em que o estado do treinamento foi salvo.

Para usá-lo, forneça uma instância de `tf.keras.callbacks.BackupAndRestore` na chamada a `Model.fit`.

Com `MultiWorkerMirroredStrategy`, se um worker for interrompido, todo o cluster vai parar até que o worker interrompido seja reiniciado. Os outros workers também reiniciarão, e o worker interrompido retornará ao cluster. Depois, cada worker lerá o arquivo de checkpoint salvo anteriormente e assumirá o estado anterior, permitindo assim que o cluster volte a ficar sincronizado. Em seguida, o treinamento continuará. O estado do iterador do dataset distribuído será reinicializado, não restaurado.

O callback `BackupAndRestore` usa `CheckpointManager` para salvar e restaurar o estado do treinamento, o que gera um arquivo chamado checkpoint que monitora os checkpoints existentes em conjunto com o último. Por esse motivo, `backup_dir` não deve ser reutilizado para armazenar outros checkpoints, evitando assim um conflito de nomes.

Atualmente, o callback `BackupAndRestore` tem suporte ao treinamento em um único worker sem estratégia `MirroredStrategy` e ao treinamento multiworker com `MultiWorkerMirroredStrategy`.

Veja abaixo dois exemplos, um para treinamento multiworker e outro para treinamento em um único worker:

In [ ]:
# Multi-worker training with `MultiWorkerMirroredStrategy`
# and the `BackupAndRestore` callback. The training state 
# is backed up at epoch boundaries by default.

callbacks = [tf.keras.callbacks.BackupAndRestore(backup_dir='/tmp/backup')]
with strategy.scope():
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()
multi_worker_model.fit(multi_worker_dataset,
                       epochs=3,
                       steps_per_epoch=70,
                       callbacks=callbacks)

Se o argumento `save_freq` do callback `BackupAndRestore` for definido como `'epoch'`, será feito backup do modelo após cada época.

In [ ]:
# The training state is backed up at epoch boundaries because `save_freq` is
# set to `epoch`.

callbacks = [tf.keras.callbacks.BackupAndRestore(backup_dir='/tmp/backup')]
with strategy.scope():
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()
multi_worker_model.fit(multi_worker_dataset,
                       epochs=3,
                       steps_per_epoch=70,
                       callbacks=callbacks)


Observação: o trecho de código seguinte usa recursos disponíveis somente no `tf-nightly` até o lançamento do TensorFlow 2.10.

Se o argumento `save_freq` do callback `BackupAndRestore` for definido como um valor inteiro maior do que `0`, é feito backup do modelo após cada `save_freq` lotes.

In [ ]:
# The training state is backed up at every 30 steps because `save_freq` is set
# to an integer value of `30`.

callbacks = [tf.keras.callbacks.BackupAndRestore(backup_dir='/tmp/backup', save_freq=30)]
with strategy.scope():
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()
multi_worker_model.fit(multi_worker_dataset,
                       epochs=3,
                       steps_per_epoch=70,
                       callbacks=callbacks)

Se você analisar o diretório de `backup_dir` especificado em `BackupAndRestore`, poderá notar alguns arquivos de checkpoint gerados temporariamente. Esses arquivos são necessários para recuperar as instâncias perdidas anteriormente e serão removidos pela biblioteca no final de `Model.fit` ao sair do treinamento com êxito.

Observação: atualmente, o callback `BackupAndRestore` tem suporte somente ao modo adiantado (eager). No modo grafo, considere usar `Model.save`/`tf.saved_model.save` e `tf.keras.models.load_model` para salvar e restaurar modelos, respectivamente, conforme descrito na seção *Como salvar e restaurar o modelo* acima e fornecendo `initial_epoch` no `Model.fit` durante o treinamento.

## Recursos adicionais

1. O guia [Treinamento distribuído no TensorFlow](../../guide/distributed_training.ipynb) apresenta uma visão geral das estratégias de distribuição disponíveis.
2. O tutorial [Loop de treinamento personalizado com o Keras e MultiWorkerMirroredStrategy](multi_worker_with_ctl.ipynb) mostra como usar `MultiWorkerMirroredStrategy` com o Keras e um loop de treinamento personalizado.
3. Confira os [Modelos oficiais](https://github.com/tensorflow/models/tree/master/official), muitos dos quais podem ser configurados para executarem diversas estratégias de distribuição.
4. O guia [Desempenho melhor com tf.function](../../guide/function.ipynb) apresenta informações sobre outras estratégias e ferramentas, como o [TensorFlow Profiler](../../guide/profiler.md), que você pode usar para otimizar o desempenho dos seus modelos do TensorFlow.